# Combine multiple QSVR trained with small data

In [47]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time
from qa_summer.QSVR import QSVR
from dimod import ExactSolver
from sklearn.preprocessing import QuantileTransformer
from sklearn.model_selection import train_test_split
from utils import nb_utils
from joblib import dump, load
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
import datetime
from random import randint, random
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVR
import neal #import to use simulated annealing sampler
from dwave.system import LazyFixedEmbeddingComposite, DWaveSampler #import to select specific sampler

In [48]:
experiment_name = ''
save = True
num_splits = 10
train_size = 80 # divisible by 20
#date = datetime.datetime.now().strftime("_%Y_%m_%d-%I:%M:%S.%f_%p")
#experiment_name = experiment_name + date

In [49]:
#load and scale data
# load data
df_info = nb_utils.get_df_info('mlpf')
df = pd.read_csv(df_info['df_path'])
df = df.drop(df[df.loss_99 == df.loss_99.max()].index)

# Select features
curve = nb_utils.get_curve(df_info=df_info, known_curve=0.25, df=df)
X = curve[:,[i for i in range(0,curve.shape[1],2)]]

# Prediction target
y = nb_utils.get_target(df_info,df)

# Scale data
x_scaler = QuantileTransformer(n_quantiles=50,random_state=0)
X = x_scaler.fit_transform(X)
y_scaler =  QuantileTransformer(n_quantiles=50,random_state=0)
y = y_scaler.fit_transform(y.reshape(-1, 1)).ravel()

In [50]:
'''
WARNING: THIS CELL SENDS PROBLEMS TO D-WAVE MULTIPLE TIMES
REMEMBRER D-WAVE AVALIABLE TIME IS LIMITED
'''
rs = randint(0, 2**30)
all_r2 = np.zeros(num_splits)
for i in range(num_splits):
	# train test split
	X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=rs+i)
	X_train, y_train = X_train[:train_size], y_train[:train_size]

	# split the training set in subsets of 20 elements an use each one to train a different
	total = int(train_size/20)
	y_preds = []
	weights = np.zeros(total)
	model = QSVR.QSVR() # instantiate Moved outside the loop so that sub-QSVRs that will be combined can use the same embedding saving local time
	for j in range(total):
		X_train_j, y_train_j = X_train[20*j:20*(j+1)], y_train[20*j:20*(j+1)]
		#model = QSVR.QSVR() # instantiate Moved outside the loop so that sub-QSVRs that will be combined can use the same embedding saving local time
		#RUN ON D-WAVE
		#set sampler
		#sampler = LazyFixedEmbeddingComposite(DWaveSampler(region='na-west-1', solver='Advantage_system6.1'))
		sampler = neal.SimulatedAnnealingSampler() #Simulated sampler doesnt use quantum time
		model.fit(X_train_j, y_train_j,
			K = 3, B = 0.5,
			epsilon = 0.02, k0 = 0.005,
			xi=0.01, n_samples = 20,
			#num_reads = 5000,
			num_reads=1000,
			random_seed=rs+i+j,
			n_samples_for_gamma_and_C_optimizations=0,
			gamma=0.1, C=67.61,
			use_custom_chainstrength=True,
			chain_mult=10,
			#anneal_time=40,
			sampler = sampler
		)
		if save: nb_utils.save_qsvr(model, 'qsvr_attrs_'+experiment_name+'_rs'+str(rs)+'_i'+str(i)+'_j'+str(j)) # save QSVR for further predictions
		remove = [x for x in range(20*j,20*(j+1))]
		X_partial_test, y_partial_test = np.delete(X_train, remove, 0), np.delete(y_train, remove, 0)
		y_partial_pred = model.predict(X_partial_test)
		partial_r2 = np.zeros(y_partial_pred.shape[0])
		for k in range(y_partial_pred.shape[0]):
			partial_r2[k] = r2_score(y_partial_pred[k],y_partial_test)
		weights[j]=partial_r2.max()
		if weights[j] < 0: weights[j] = 0 
		method = partial_r2.argmax()
		y_preds.append(model.predict(X_test)[method])
		
	#combine predictions
	y_pred = np.zeros(y_test.shape[0])
	for k in range(total):
		y_pred = y_pred + weights[k]*y_preds[k]
	y_pred = y_pred / weights.sum()

	all_r2[i] = r2_score(y_pred,y_test)

Creating the QUBO Q matrix of size (120, 120)
Extracting nodes and couplers from Q
The problem has 120 nodes and 7140 couplers
-> Setting sampler to: LazyFixedEmbeddingComposite(DWaveSampler()) 


2022-08-25 16:41:45,691 dwave.cloud.client.base INFO MainThread Fetching available regions from the Metadata API at 'https://cloud.dwavesys.com/metadata/v1/'
2022-08-25 16:41:47,038 dwave.cloud.client.base INFO MainThread Using region metadata: [Region(code='na-west-1', name='North America', endpoint='https://na-west-1.cloud.dwavesys.com/sapi/v2/'), Region(code='eu-central-1', name='Europe', endpoint='https://eu-central-1.cloud.dwavesys.com/sapi/v2/')]
2022-08-25 16:41:47,065 dwave.cloud.client.base INFO MainThread Requested a solver that best matches feature filters={}
2022-08-25 16:41:47,067 dwave.cloud.client.base INFO MainThread Fetching solvers according to filters={}, order_by='-num_active_qubits'
2022-08-25 16:41:47,068 dwave.cloud.client.base INFO MainThread Fetching definitions of all available solvers
2022-08-25 16:41:48,268 dwave.cloud.client.base INFO MainThread Received solver data for 1 solver(s).
2022-08-25 16:41:48,344 dwave.cloud.client.base INFO MainThread Adding solv

Running with 120 nodes and 7140 couplers
Chain Breaks mean 0.013491666666666666
Creating the QUBO Q matrix of size (120, 120)
Extracting nodes and couplers from Q
The problem has 120 nodes and 7140 couplers
Running with 120 nodes and 7140 couplers
Chain Breaks mean 0.014733333333333333
Creating the QUBO Q matrix of size (120, 120)
Extracting nodes and couplers from Q
The problem has 120 nodes and 7140 couplers
Running with 120 nodes and 7140 couplers
Chain Breaks mean 0.014116666666666668
Creating the QUBO Q matrix of size (120, 120)
Extracting nodes and couplers from Q
The problem has 120 nodes and 7140 couplers
Running with 120 nodes and 7140 couplers
Chain Breaks mean 0.015141666666666666
Creating the QUBO Q matrix of size (120, 120)


2022-08-25 16:46:38,058 dwave.cloud.client.base INFO MainThread Using region metadata: [Region(code='na-west-1', name='North America', endpoint='https://na-west-1.cloud.dwavesys.com/sapi/v2/'), Region(code='eu-central-1', name='Europe', endpoint='https://eu-central-1.cloud.dwavesys.com/sapi/v2/')]
2022-08-25 16:46:38,129 dwave.cloud.client.base INFO MainThread Requested a solver that best matches feature filters={}
2022-08-25 16:46:38,130 dwave.cloud.client.base INFO MainThread Fetching solvers according to filters={}, order_by='-num_active_qubits'
2022-08-25 16:46:38,132 dwave.cloud.client.base INFO MainThread Fetching definitions of all available solvers


Extracting nodes and couplers from Q
The problem has 120 nodes and 7140 couplers
-> Setting sampler to: LazyFixedEmbeddingComposite(DWaveSampler()) 


...

Running with 120 nodes and 7140 couplers
Chain Breaks mean 0.018991666666666667
Creating the QUBO Q matrix of size (120, 120)
Extracting nodes and couplers from Q
The problem has 120 nodes and 7140 couplers
Running with 120 nodes and 7140 couplers
Chain Breaks mean 0.019399999999999997
Creating the QUBO Q matrix of size (120, 120)
Extracting nodes and couplers from Q
The problem has 120 nodes and 7140 couplers
Running with 120 nodes and 7140 couplers
Chain Breaks mean 0.019008333333333332
Creating the QUBO Q matrix of size (120, 120)
Extracting nodes and couplers from Q
The problem has 120 nodes and 7140 couplers
Running with 120 nodes and 7140 couplers
Chain Breaks mean 0.018291666666666668
Creating the QUBO Q matrix of size (120, 120)


2022-08-25 17:18:33,497 dwave.cloud.client.base INFO MainThread Using region metadata: [Region(code='na-west-1', name='North America', endpoint='https://na-west-1.cloud.dwavesys.com/sapi/v2/'), Region(code='eu-central-1', name='Europe', endpoint='https://eu-central-1.cloud.dwavesys.com/sapi/v2/')]
2022-08-25 17:18:33,589 dwave.cloud.client.base INFO MainThread Requested a solver that best matches feature filters={}
2022-08-25 17:18:33,591 dwave.cloud.client.base INFO MainThread Fetching solvers according to filters={}, order_by='-num_active_qubits'
2022-08-25 17:18:33,592 dwave.cloud.client.base INFO MainThread Fetching definitions of all available solvers


Extracting nodes and couplers from Q
The problem has 120 nodes and 7140 couplers
-> Setting sampler to: LazyFixedEmbeddingComposite(DWaveSampler()) 


2022-08-25 17:18:34,073 dwave.cloud.client.base INFO MainThread Received solver data for 1 solver(s).
2022-08-25 17:18:34,143 dwave.cloud.client.base INFO MainThread Adding solver StructuredSolver(id='Advantage_system5.2')
2022-08-25 17:18:34,145 dwave.cloud.client.base INFO MainThread Filtered solvers=[StructuredSolver(id='Advantage_system5.2')]


Running with 120 nodes and 7140 couplers
Chain Breaks mean 0.019266666666666665
Creating the QUBO Q matrix of size (120, 120)
Extracting nodes and couplers from Q
The problem has 120 nodes and 7140 couplers
Running with 120 nodes and 7140 couplers
Chain Breaks mean 0.018299999999999997
Creating the QUBO Q matrix of size (120, 120)
Extracting nodes and couplers from Q
The problem has 120 nodes and 7140 couplers
Running with 120 nodes and 7140 couplers
Chain Breaks mean 0.019700000000000002
Creating the QUBO Q matrix of size (120, 120)
Extracting nodes and couplers from Q
The problem has 120 nodes and 7140 couplers
Running with 120 nodes and 7140 couplers
Chain Breaks mean 0.019683333333333337


In [51]:
print(f'All r2 metrics:\nmean = {all_r2.mean()} \nmax = {all_r2.max()} \nmin = {all_r2.min()} \nmedian = {np.median(all_r2)} \nstd = {all_r2.std()}')
print(all_r2)

All r2 metrics:
mean = 0.8990778217229403 
max = 0.9224889002941077 
min = 0.8745733261455606 
median = 0.9017957908205764 
std = 0.015316481064391176
[0.89524831 0.90422806 0.87794839 0.91828191 0.87457333 0.9224889
 0.90315186 0.90976909 0.90043972 0.88464864]


In [52]:
dump(all_r2, 'all_r2_quantum.joblib')
loaded_r2 = load('all_r2_80.joblib')
print(f'All r2 metrics:\nmean = {loaded_r2.mean()} \nmax = {loaded_r2.max()} \nmin = {loaded_r2.min()} \nmedian = {np.median(loaded_r2)} \nstd = {loaded_r2.std()}')
loaded_r2

All r2 metrics:
mean = 0.9218500066090553 
max = 0.9437759777269824 
min = 0.9008821879200034 
median = 0.9166478859797293 
std = 0.012006228408819758


array([0.93289407, 0.93050689, 0.914473  , 0.93269036, 0.90088219,
       0.94377598, 0.91509265, 0.91523155, 0.91806422, 0.91488917])

### Old part for studying different combination techniques

In [ ]:
'''
WARNING: THIS CELL SENDS PROBLEMS TO D-WAVE MULTIPLE TIMES
REMEMBRER D-WAVE AVALIABLE TIME IS LIMITED
'''
rs = randint(0, 2**30)
all_r2_1 = np.zeros((num_splits, 7))
all_r2_2 = np.zeros((num_splits, 7))
all_r2_3 = np.zeros((num_splits, 7))
all_r2_4 = np.zeros((num_splits, 7))
for i in range(num_splits):
	# train test split
	X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=rs+i)
	X_train, y_train = X_train[:train_size], y_train[:train_size]

	# split the training set in subsets of 20 elements an use each one to train a different
	qsvr_models_dict = []
	#qsvr_models = []
	#y_preds = []
	total = int(train_size/20)
	weights = np.zeros(total)
	for j in range(total):
		X_train_j, y_train_j = X_train[20*j:20*(j+1)], y_train[20*j:20*(j+1)]
		model = QSVR.QSVR() # instantiate
		#RUN ON D-WAVE
		#set sampler
		#sampler = LazyFixedEmbeddingComposite(DWaveSampler(region='na-west-1', solver='Advantage_system6.1'))
		sampler = neal.SimulatedAnnealingSampler()
		model.fit(X_train_j, y_train_j,
			K = 3, B = 0.5,
			epsilon = 0.02, k0 = 0.005,
			xi=0.01, n_samples = 20,
			#num_reads = 5000,
			num_reads=1000,
			random_seed=rs+i+j,
			n_samples_for_gamma_and_C_optimizations=0,
			gamma=0.1, C=67.61,
			use_custom_chainstrength=True,
			chain_mult=10,
			#anneal_time=40,
			sampler=sampler
		)
		if save: nb_utils.save_qsvr(model, 'qsvr_attrs_'+experiment_name+'_rs'+str(rs)+'_i'+str(i)+'_j'+str(j)) # save QSVR for further predictions
		qsvr_models_dict.append(nb_utils.qsvr_to_pred_dict(model))

		#qsvr_models.append(model)
		#y_preds.append(qsvr_models[j].predict(X_test))
		remove = [x for x in range(20*j,20*(j+1))]
		X_partial_test, y_partial_test = np.delete(X_train, remove, 0), np.delete(y_train, remove, 0)
		y_partial_pred = model.predict(X_partial_test)
		partial_r2 = np.zeros(y_partial_pred.shape[0])
		for k in range(y_partial_pred.shape[0]):
			partial_r2[k] = r2_score(y_partial_pred[k],y_partial_test)
		weights[j]=partial_r2.mean()
		
	# combine trained models to do predictions
	X_train_reshaped = qsvr_models_dict[0]['X_train_reshaped'][:]
	Y_train = qsvr_models_dict[0]['Y_train']
	all_alphas = qsvr_models_dict[0]['all_alphas']
	for j in range(1,int(train_size/20)):
		X_train_reshaped.extend(qsvr_models_dict[j]['X_train_reshaped'])
		Y_train = np.concatenate((Y_train, qsvr_models_dict[j]['Y_train']))
		all_alphas = np.concatenate((all_alphas,qsvr_models_dict[j]['all_alphas']),axis=2)
   
	combined_model_dict = {}
	combined_model_dict['X_train_reshaped'] = X_train_reshaped
	combined_model_dict['Y_train'] = Y_train
	combined_model_dict['all_alphas'] = all_alphas
	combined_model_dict['B'] = qsvr_models_dict[0]['B']
	combined_model_dict['K'] = qsvr_models_dict[0]['K']
	combined_model_dict['epsilon'] = qsvr_models_dict[0]['epsilon']
	combined_model_dict['best_gamma'] = qsvr_models_dict[0]['best_gamma']
	combined_model_dict['best_C'] = qsvr_models_dict[0]['best_C']
	combined_model_dict['change_to_logarithmic'] = qsvr_models_dict[0]['change_to_logarithmic']

	combined_model = QSVR.QSVR() 
	# set the QSVR attributes needed for predictions
	for attr in combined_model_dict.keys():
		setattr(combined_model, attr, combined_model_dict[attr])
	
	y_pred = combined_model.predict(X_test)

	# make use of the prediction
	r2_1 = []
	for k in range(y_pred.shape[0]):
		r2_1.append(r2_score(y_pred[k],y_test))
	
	#second combination technique
	y_pred = np.zeros((7,y_test.shape[0]))
	for pred_model_dict in qsvr_models_dict:
		pred_model = QSVR.QSVR()
		for attr in pred_model_dict.keys():
			setattr(pred_model, attr, pred_model_dict[attr])
		y_pred = y_pred + pred_model.predict(X_test)
	y_pred = y_pred / float(total)
	r2_2 = []
	for k in range(y_pred.shape[0]):
		r2_2.append(r2_score(y_pred[k],y_test))

	#third combination technique
	y_pred = np.zeros((7,y_test.shape[0]))
	idx = 0
	for pred_model_dict in qsvr_models_dict:
		pred_model = QSVR.QSVR()
		for attr in pred_model_dict.keys():
			setattr(pred_model, attr, pred_model_dict[attr])
		y_pred = y_pred + weights[idx]*pred_model.predict(X_test)
		idx = idx +1
	y_pred = y_pred / weights.sum()
	r2_3 = []
	for k in range(y_pred.shape[0]):
		r2_3.append(r2_score(y_pred[k],y_test))
	
	#third combination technique
	y_pred = np.zeros((7,y_test.shape[0]))
	idx = 0
	weights_tr = 10**weights
	for pred_model_dict in qsvr_models_dict:
		pred_model = QSVR.QSVR()
		for attr in pred_model_dict.keys():
			setattr(pred_model, attr, pred_model_dict[attr])
		y_pred = y_pred + weights[idx]*pred_model.predict(X_test)
		idx = idx +1
	y_pred = y_pred / weights.sum()
	r2_4 = []
	for k in range(y_pred.shape[0]):
		r2_4.append(r2_score(y_pred[k],y_test))
	
	
	for j in range(7):
		all_r2_1[i,j] = r2_1[j]
		all_r2_2[i,j] = r2_2[j]
		all_r2_3[i,j] = r2_3[j]
		all_r2_4[i,j] = r2_4[j]

results_1 = {
	'scores norm' : all_r2_1[:,0],
	'scores softmax' : all_r2_1[:,1],
	'scores lc norm' : all_r2_1[:,2],
	'scores lc softmax' : all_r2_1[:,3],
	'best set of alphas' : all_r2_1[:,4],
	'simple mean' : all_r2_1[:,5],
	'min energy' : all_r2_1[:,6]
}
results_2 = {
	'scores norm' : all_r2_2[:,0],
	'scores softmax' : all_r2_2[:,1],
	'scores lc norm' : all_r2_2[:,2],
	'scores lc softmax' : all_r2_2[:,3],
	'best set of alphas' : all_r2_2[:,4],
	'simple mean' : all_r2_2[:,5],
	'min energy' : all_r2_2[:,6]
}

results_3 = {
	'scores norm' : all_r2_3[:,0],
	'scores softmax' : all_r2_3[:,1],
	'scores lc norm' : all_r2_3[:,2],
	'scores lc softmax' : all_r2_3[:,3],
	'best set of alphas' : all_r2_3[:,4],
	'simple mean' : all_r2_3[:,5],
	'min energy' : all_r2_3[:,6]
}

results_4 = {
	'scores norm' : all_r2_4[:,0],
	'scores softmax' : all_r2_4[:,1],
	'scores lc norm' : all_r2_4[:,2],
	'scores lc softmax' : all_r2_4[:,3],
	'best set of alphas' : all_r2_4[:,4],
	'simple mean' : all_r2_4[:,5],
	'min energy' : all_r2_4[:,6]
}

In [55]:
dump(results_2, 'results_2_rs'+str(rs)+'.joblib')

['results_2_rs561525581.joblib']

In [51]:
f_r2 = all_r2_2.flatten()
print(f'All r2 metrics:\nmean = {f_r2.mean()} \nmax = {f_r2.max()} \nmin = {f_r2.min()} \nmedian = {np.median(f_r2)} \nstd = {f_r2.std()}')

All r2 metrics:
mean = 0.9230285516029251 
max = 0.9371091309109142 
min = 0.8988907982478825 
median = 0.9244619471846305 
std = 0.010157407864016952


In [52]:
f_r2 = all_r2_3.flatten()
print(f'All r2 metrics:\nmean = {f_r2.mean()} \nmax = {f_r2.max()} \nmin = {f_r2.min()} \nmedian = {np.median(f_r2)} \nstd = {f_r2.std()}')

All r2 metrics:
mean = 0.9232230674066298 
max = 0.9376890106799478 
min = 0.8991614267546353 
median = 0.9245566573373835 
std = 0.010165078969416217


In [53]:
f_r2= all_r2_4.flatten()
print(f'All r2 metrics:\nmean = {f_r2.mean()} \nmax = {f_r2.max()} \nmin = {f_r2.min()} \nmedian = {np.median(f_r2)} \nstd = {f_r2.std()}')

All r2 metrics:
mean = 0.9232230674066298 
max = 0.9376890106799478 
min = 0.8991614267546353 
median = 0.9245566573373835 
std = 0.010165078969416217


In [46]:
results_3

{'scores norm': array([0.89254972]),
 'scores softmax': array([0.90356923]),
 'scores lc norm': array([0.89253391]),
 'scores lc softmax': array([0.90330851]),
 'best set of alphas': array([0.90356923]),
 'simple mean': array([0.88730293]),
 'min energy': array([0.88289149])}